# 기본 정보 입력

In [1]:
# 패키지 불러오기
import openai

In [2]:
# API 키 지정하여 클라이언트 선언하기
OPENAI_API_KEY="API_KEY"
client = openai.OpenAI(api_key = OPENAI_API_KEY)

## Knowledge 로 사용할 파일 업로드

In [3]:
file = client.files.create(
  file=open("축구규칙정리.pdf", "rb"),
  purpose='assistants'
)
print(file.id)

file-xh5nP3jEKdKbFOGK8ocPqICr


## 내가 업로드한 파일 목록 확인하기

In [4]:

file_list = client.files.list()
print(file_list)

SyncPage[FileObject](data=[FileObject(id='file-gmm02rIMFkKpNRr2wrj3sV5n', bytes=30066107, created_at=1702644125, filename='soccer_rule_book.pdf', object='file', purpose='assistants', status='processed', status_details=None)], object='list', has_more=False)


In [5]:
# 특정 파일 ID 가져오기

file_id = file_list.data[0].id
print(file_id)

file-gmm02rIMFkKpNRr2wrj3sV5n


# Create Assistant
- instructions: 어시스턴트와 모델이 어떻게 행동하거나 응답해야 하는지 알려줍니다.
- model: 미세 조정된 모델을 포함하여 모든 GPT-3.5 또는 GPT-4 모델을 지정할 수 있습니다. 검색 도구에는 gpt-3.5-turbo-1106 및 gpt-4-1106-preview 모델이 필요합니다.
- tools: API는 OpenAI에서 빌드 및 호스팅하는 코드 인터프리터와 웹 검색을 지원합니다. 또한 함수 호출 기능과 유사한 동작으로 사용자 정의 함수 서명을 정의할 수 있습니다


In [4]:
instruction = '''
[목적]
이 GPT는 축구 규칙을 상세히 설명해주는 챗봇입니다.

[규칙]
1. 사용자가 축구 규칙에 대해 질문하면 업로드된 파일에서 해당 내용을 찾아 자세히 답변합니다.
2. 파일안에서 마땅한 답을 찾을 수 없거나 축구 규칙에 관한 질문이 아니면 "축구 규칙에 관한 질문만 부탁해요^^" 라고 답해주세요.
3. 답변의 형태는 아래 예시와 같이 해주세요
예시)
-  질문 : 질문 내용
-  답변 : 답변내용 
4. 모든 질문에 한국어로 답변해주세요.
'''

In [5]:
assistant = client.beta.assistants.create(
  name = "축구 규칙 설명 챗봇",
  instructions=instruction,
  model="gpt-3.5-turbo-1106",
  tools=[{"type": "retrieval"}],
  file_ids=[file.id]
)

print(assistant.id)

asst_GHdKitF0kR55ZyVnGKr8QaCB


## 생성한 Assistant 업데이트 하기

In [ ]:
# Run this only if you need to update the configuration of the assistant
assistant = client.beta.assistants.update(
  assistant.id,
  instructions=instruction,
  model="gpt-4-1106-preview",
  #model="gpt-3.5-turbo-1106",
  tools=[{"type": "retrieval"}],
  file_ids=[file.id]
)

## 기존 Assistant 불러오기

In [9]:
# 나의 Assistant 목록 확인
my_assistants = client.beta.assistants.list(
    order="desc",
    limit="20",
)
print(my_assistants.data)

[Assistant(id='asst_2zHoFZNdrDxlqksAmPTRAweO', created_at=1709470447, description=None, file_ids=['file-5u9FN54oWlCC1FpPi26IMAMY'], instructions='\n[목적]\n이 GPT는 축구 규칙을 상세히 설명해주는 챗봇입니다.\n\n[규칙]\n1. 사용자가 축구 규칙에 대해 질문하면 업로드된 파일에서 해당 내용을 찾아 자세히 답변합니다.\n2. 파일안에서 마땅한 답을 찾을 수 없거나 축구 규칙에 관한 질문이 아니면 "축구 규칙에 관한 질문만 부탁해요^^" 라고 답해주세요.\n3. 답변의 형태는 아래 예시와 같이 해주세요\n예시)\n-  질문 : 질문 내용\n-  답변 : 답변내용 \n4. 모든 질문에 한국어로 답변해주세요.\n', metadata={}, model='gpt-3.5-turbo-1106', name='축구 규칙 설명 챗봇', object='assistant', tools=[ToolRetrieval(type='retrieval')])]


In [4]:
assistant = client.beta.assistants.retrieve("asst_KoH4sLsw9B4vwjsKVbpmKy1l")
print(assistant)

Assistant(id='asst_KoH4sLsw9B4vwjsKVbpmKy1l', created_at=1702534045, description=None, file_ids=['file-wRakvw7TiO1b2rNUrh7kSsGc'], instructions="\nYou're a soccer expert explaining the rules of the game\n\nRules:\n1. when a user asks you a question about the rules of soccer, you find it in the uploaded file and answer it in detail.\n2. if it's not in the file, say you're not sure.\n3. answer all questions in Korean.\n", metadata={}, model='gpt-3.5-turbo-1106', name='축구 규칙 설명봇t', object='assistant', tools=[ToolRetrieval(type='retrieval')])


# Tread

## Tread 생성하기

In [6]:
thread = client.beta.threads.create()
print(thread)

Thread(id='thread_NDE00aRkYxvU5rLaNAE9izbA', created_at=1709622201, metadata={}, object='thread')


## Tread 에 메세지 넣기

In [7]:
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="축구장의 크기는?"
)
print(message)

ThreadMessage(id='msg_H8Hok2ZB5Xd5JlpXpBz5jdsO', assistant_id=None, content=[MessageContentText(text=Text(annotations=[], value='축구장의 크기는?'), type='text')], created_at=1709622203, file_ids=[], metadata={}, object='thread.message', role='user', run_id=None, thread_id='thread_NDE00aRkYxvU5rLaNAE9izbA')


# Assistant Run 하기

In [9]:
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id)

In [10]:
import time

while run.status not in ["completed", "failed"]:
  run = client.beta.threads.runs.retrieve(
    thread_id= thread.id,
    run_id= run.id
  )
  print(run.status)
  time.sleep(5)

in_progress
in_progress
completed


# 결과확인(Run complete 후)

In [11]:
messages = client.beta.threads.messages.list(
  thread_id=thread.id
)

In [12]:
for each in messages:
  print(each.role + ": "+each.content[0].text.value)
  print("=========")

assistant: 축구장의 크기는 국제 경기를 위한 경우와 일반적인 대회를 위한 경우로 구분됩니다.

- 국제 경기를 위한 크기:
  - 터치라인: 최소 100m (110 야드) - 최대 110m (120 야드)
  - 골라인: 최소 64m (70 야드) - 최대 75m (80 야드)

- 대회를 위한 크기:
  - 터치라인: 최소 90m (100 야드) - 최대 120m (130 야드)
  - 골라인: 최소 45m (50 야드) - 최대 90m (100 야드)

대회 주최 측은 위의 범위 내에서 골라인과 터치라인의 길이를 결정할 수 있습니다【7†출처】.
user: 축구장의 크기는?


# 질문 추가하기

In [14]:
# 메세지 추가
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="오프사이드 룰에 대해 설명해줘"
)

# 메세지 리스트 가져오기
messages = client.beta.threads.messages.list(
  thread_id=thread.id
)

#메세지 프린트
for each in messages:
  print(each.role + ": "+each.content[0].text.value)
  print("=========")

user: 오프사이드 룰에 대해 설명해줘?
assistant: 축구장의 크기는 국제 경기를 위한 경우와 일반적인 대회를 위한 경우로 구분됩니다.

- 국제 경기를 위한 크기:
  - 터치라인: 최소 100m (110 야드) - 최대 110m (120 야드)
  - 골라인: 최소 64m (70 야드) - 최대 75m (80 야드)

- 대회를 위한 크기:
  - 터치라인: 최소 90m (100 야드) - 최대 120m (130 야드)
  - 골라인: 최소 45m (50 야드) - 최대 90m (100 야드)

대회 주최 측은 위의 범위 내에서 골라인과 터치라인의 길이를 결정할 수 있습니다【7†출처】.
user: 축구장의 크기는?


In [15]:
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id,)

while run.status not in ["completed", "failed"]:

  run = client.beta.threads.runs.retrieve(
    thread_id= thread.id,
    run_id= run.id)

  print(run.status)

  time.sleep(5)

in_progress
in_progress
completed


In [16]:
# 메세지 리스트 가져오기
messages = client.beta.threads.messages.list(
  thread_id=thread.id
)

#메세지 프린트
for each in messages:
  print(each.role + ": "+each.content[0].text.value)
  print("=========")

assistant: 오프사이드 룰은 다음과 같은 내용을 포함합니다:

1. 오프사이드 위치:
   - 오프사이드 위치에 있는 것만으로는 반칙이 아닙니다. 오프사이드 위치에 있는 선수는 특정 조건을 충족해야 오프사이드라고 판단됩니다.
   - 대략적으로 말하면, 상대방 골라인에 더 가까이 있고, 볼과 마지막으로 부딪힌 상대팀 수비수보다 더 앞선 위치에 있는 선수가 오프사이드 위치에 있다고 판단됩니다.

2. 오프사이드 반칙:
   - 오프사이드 위치에 있는 선수가 팀 동료에 의해 볼이 플레이되거나 터치된 순간에 적극적으로 플레이에 관여하거나 상대 선수를 방해할 때에만 처벌을 받습니다.
   - 상대팀 선수의 시선을 가림으로써 플레이를 방해하거나, 명백하게 상대 선수의 플레이를 방해하는 행위를 하거나, 볼을 목적으로 상대 선수에게 도전하는 행위 등이 이에 해당합니다.

상세한 내용은 규칙서에 기재되어 있으며, 위의 내용은 그 일부에 불과합니다【6†출처】.
user: 오프사이드 룰에 대해 설명해줘?
assistant: 축구장의 크기는 국제 경기를 위한 경우와 일반적인 대회를 위한 경우로 구분됩니다.

- 국제 경기를 위한 크기:
  - 터치라인: 최소 100m (110 야드) - 최대 110m (120 야드)
  - 골라인: 최소 64m (70 야드) - 최대 75m (80 야드)

- 대회를 위한 크기:
  - 터치라인: 최소 90m (100 야드) - 최대 120m (130 야드)
  - 골라인: 최소 45m (50 야드) - 최대 90m (100 야드)

대회 주최 측은 위의 범위 내에서 골라인과 터치라인의 길이를 결정할 수 있습니다【7†출처】.
user: 축구장의 크기는?


# Assistants / File 삭제

In [22]:
# 특정 assistant 에서만 제거하고 싶으면
response = client.beta.assistants.files.delete( 
  assistant_id=assistant.id, 
  file_id=file.id
)
print(response)

TypeError: Files.delete() missing 1 required keyword-only argument: 'assistant_id'

In [19]:
# 업로드 파일 삭제
response = client.files.delete(file.id)
print(response)
# assistant 삭제
response = client.beta.assistants.delete(assistant.id)
print(response)

FileDeleted(id='file-xh5nP3jEKdKbFOGK8ocPqICr', deleted=True, object='file')
AssistantDeleted(id='asst_GHdKitF0kR55ZyVnGKr8QaCB', deleted=True, object='assistant.deleted')


In [13]:
# assistant 삭제
response = client.beta.assistants.delete(assistant.id)

AssistantDeleted(id='asst_90ls4Pbf8Aa7tWEMUnsQbYw2', deleted=True, object='assistant.deleted')
